In [1]:
print('ok')

ok


In [2]:
%pwd

'/workspaces/MedicalChatbot/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/workspaces/MedicalChatbot'

In [5]:
from langchain_community.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Pinecone


In [6]:
def load_pdf_files(directory):
    loader=DirectoryLoader(directory, glob="**/*.pdf" , loader_cls=PyPDFLoader)

    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_files("data")

In [8]:
len(extracted_data)

637

In [9]:
from typing import List
from langchain.schema import Document
def filter(docs: List[Document]) -> List[Document]:
    """
    Load all PDF files from a specified directory.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={
                    "source": src}
            )
        )
    return minimal_docs



In [10]:
minimal_docs = filter(extracted_data)

In [11]:
def split_documents(documents):
    """
    Split documents into smaller chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=30,
    )
    
    return text_splitter.split_documents(documents)

In [12]:
text_chunks = split_documents(minimal_docs)
len(text_chunks)

5875

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)

texts = ["This is an example sentence.", "LangChain is cool!"]
embeddings = embedding.embed_documents(texts)

print(f"Embedding dimension: {len(embeddings[0])}")
print(embeddings)

/tmp/ipykernel_10016/2336940152.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dimension: 384
[[0.09812458604574203, 0.06781266629695892, 0.06252316385507584, 0.09508480876684189, 0.03664763644337654, -0.003984622657299042, 0.007477528881281614, -0.013231469318270683, 0.06288369745016098, 0.02249552495777607, 0.0726957842707634, -0.031274255365133286, 0.04635513201355934, -0.012554480694234371, 0.047814734280109406, -0.004910331219434738, 0.04941995069384575, -0.06410931050777435, -0.09696582704782486, 0.03288879990577698, 0.05410442501306534, 0.03532864525914192, 0.033050574362277985, 0.014699426479637623, -0.03343065083026886, -0.025615906342864037, -0.05079213157296181, 0.07325457036495209, 0.11027400195598602, -0.029661819338798523, -0.06755705177783966, -0.03057154454290867, 0.0395602248609066, 0.04547601938247681, 0.01599622331559658, 0.038550399243831635, -0.010954082012176514, 0.08483566343784332, -0.04428711161017418, -0.006796412169933319, 0.009425660595297813, 5.071496343589388e-05, 0.001303564291447401, -0.011969784274697304, 0.0136451609432

In [14]:
embeddings

[[0.09812458604574203,
  0.06781266629695892,
  0.06252316385507584,
  0.09508480876684189,
  0.03664763644337654,
  -0.003984622657299042,
  0.007477528881281614,
  -0.013231469318270683,
  0.06288369745016098,
  0.02249552495777607,
  0.0726957842707634,
  -0.031274255365133286,
  0.04635513201355934,
  -0.012554480694234371,
  0.047814734280109406,
  -0.004910331219434738,
  0.04941995069384575,
  -0.06410931050777435,
  -0.09696582704782486,
  0.03288879990577698,
  0.05410442501306534,
  0.03532864525914192,
  0.033050574362277985,
  0.014699426479637623,
  -0.03343065083026886,
  -0.025615906342864037,
  -0.05079213157296181,
  0.07325457036495209,
  0.11027400195598602,
  -0.029661819338798523,
  -0.06755705177783966,
  -0.03057154454290867,
  0.0395602248609066,
  0.04547601938247681,
  0.01599622331559658,
  0.038550399243831635,
  -0.010954082012176514,
  0.08483566343784332,
  -0.04428711161017418,
  -0.006796412169933319,
  0.009425660595297813,
  5.071496343589388e-05,
  0

In [16]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [17]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [18]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key)

In [19]:
pc

In [ ]:
from pinecone import ServerlessSpec
index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        index_name,
        dimension=384,  # Dimension of the embeddings
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region='us-east-1'
        ),
    )

index = pc.Index(index_name)
         

In [23]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    text_chunks,
    embedding,
    index_name=index_name,
    namespace="medical-chatbot"
)

In [26]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [30]:
retrieved_docs = docsearch.similarity_search("What is Acne?")

In [31]:
retrieved_docs

[Document(id='b1a9ae72-8ccd-4a43-9c36-68e37cb94b34', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='899cdf00-79c7-4deb-be73-08f38790ccd8', metadata={'source': 'data/Medical_book.pdf'}, page_content='Acid reflux see Heartburn\nAcidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(id='37fb8162-92d5-4b52-9c0b-009b0c82a5e6', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’

In [53]:
from langchain_groq import ChatGroq

chatmodel = ChatGroq(
    model_name="llama3-70b-8192",
    temperature=0.1,
    groq_api_key=GROQ_API_KEY)

In [59]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

system_prompt =( """You are a medical chatbot that provides accurate and helpful information
 about various medical conditions, treatments, and general health advice. 
 Use the following pieces of retrieved context to answer, if you don't know the answer, say "I don't know".
 Your responses should be clear, 
concise, and based on the latest medical knowledge. Always prioritize patient safety and well-being in your answers.
Use three sentences maximum to answer the question.Also suggest some possible easy remedies or available medicines if possible.
     {context}"""
)

In [60]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [61]:
question_answer_chain  = create_stuff_documents_chain(
    llm=chatmodel,
    prompt=prompt_template)

In [62]:
rag_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=question_answer_chain,
)

In [63]:
response = rag_chain.invoke({"input": "I'm not able to breath as i'm in high altitude , travelling in sikkim , what are the possible cases?"})
print(response['answer'])

I understand your concern! At high altitudes, the air pressure is lower, and oxygen levels are decreased, which can lead to breathing difficulties. Based on your symptoms, it's possible that you may be experiencing Acute Mountain Sickness (AMS) or High-Altitude Pulmonary Edema (HAPE). 

AMS symptoms include dizziness, headache, shortness of breath, nausea, vomiting, loss of appetite, and insomnia. If you're experiencing any of these, it's essential to descend to a lower altitude and rest.

For mild AMS, you can try taking Diamox (acetazolamide) to help alleviate symptoms. However, if your symptoms worsen or you experience severe headaches, confusion, or difficulty breathing, seek medical attention immediately.

Remember to stay hydrated, avoid strenuous activities, and ascend gradually to higher altitudes to minimize the risk of altitude sickness. If you're unsure about your symptoms or condition, consult a local doctor or medical professional for guidance.
